In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - jeans_perturbations.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstring:
'''Test the application of the Jeans equation to mock data from DFs under the 
influence of various perturbations
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy
from astropy import units as apu

## Matplotlib
import matplotlib
from matplotlib import pyplot as plt

## Galpy
from galpy import orbit, potential, df

# sys.path.insert(0,'../../src/')

### Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
ro = 8.275
vo = 220.
zo = 0.0208
h = 0.7

### First prepare some DFs and draw samples

### Now calculate the Jeans quantity

The Jeans equation
$\frac{\mathrm{d} (\nu\,\overline{v^2_r})}{\mathrm{d} r} +\,\nu\,
\left(\frac{\mathrm{d} \Phi}{\mathrm{d} r}+
\frac{2\overline{v_r^2}-\overline{v_\theta^2}-\overline{v_\phi^2}}{r}\right)= 0$

This equation has units of 

$J \equiv [\ell]^{-4}[v]^{2}$

And so one way of normalizing if not using real units is to divide by 

$\mathrm{ro}^{-4}\mathrm{vo}^{2}$

In [ ]:
def calculate_spherical_jeans_quantities(orbs,pot,r_range=[0,100],n_bin=10,
    norm_by_galpy_scale_units=False,calculate_pe_with_pot=False,ro=ro,vo=vo):
    '''calculate_spherical_jeans_quantities:
    
    Calculate the quantities used in the spherical Jeans equation.
    
    Args:
        orbs (Orbits) - Orbits object containing particles / kinematic sample
        pot (Potential) - Potential object representing the gravitational 
            potential experienced by orbs
        r_range (optional, list) - Range of radii to consider, in kpc 
            [default: [0,100]]
        n_bin (optional, int) - Number of bins to use in calculating Jeans
            equation, note derivative quantities will be calculated with 
            n_bin+1 bins [default: 10]
        norm_by_galpy_scale_units (optional, bool) - If True, normalize the
            Jeans equation by galpy scale units [default: False]
        calculate_pe_with_pot (optional, bool) - If True, calculate the 
            potential at the bin centers, rather than the mean potential of the 
            orbs in the bin [default: False]
        ro (optional, float) - Distance scale in kpc [default: 8.275]
        vo (optional, float) - Velocity scale in km/s [default: 220.]
    
    Returns:
        qs (tuple) - Tuple of kinematic quantities used to calculate Jeans
            equation, output from calculate_spherical_jeans_quantities, 
            in order: dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs
    '''
    orbs = copy.deepcopy(orbs)
    orbs.turn_physical_on(ro=ro,vo=vo)
    pot = copy.deepcopy(pot)
    pot.turn_physical_on(ro=ro,vo=vo)

    ## Determine bins for kinematic properties
    
    # First need bins for derivatives, one more bin than for the data itself, 
    # since we're taking derivatives
    n_dr_bin = n_bin+1
    dr_bin_edge = np.linspace(r_range[0],r_range[1],n_dr_bin+1)
    dr_bin_cents = (dr_bin_edge[1:]+dr_bin_edge[:-1])/2
    # dr_bin_delta = dr_bin_edge[1:]-dr_bin_edge[:-1]

    # One fewer bin for data, since we're taking derivatives. The edges are 
    # the derivative bin centers
    bin_edge = copy.deepcopy(dr_bin_cents)
    bin_cents = (bin_edge[1:]+bin_edge[:-1])/2
    # bin_delta = bin_edge[1:]-bin_edge[:-1]

    # Bin the data, derivative quantities first
    nuvr2 = np.zeros_like(dr_bin_cents)
    phi = np.zeros_like(dr_bin_cents)
    # Non-derivative quantities
    nu = np.zeros_like(bin_cents)
    vr2 = np.zeros_like(bin_cents)
    vt2 = np.zeros_like(bin_cents)
    vp2 = np.zeros_like(bin_cents)

    rs = orbs.r(use_physical=True).to(apu.kpc).value
    pe = potential.evaluatePotentials(pot,orbs.R(),orbs.z(),
        use_physical=True).to(apu.km**2/apu.s**2).value
    pe_bin_cents = potential.evaluatePotentials(pot,dr_bin_cents*apu.kpc,
        0*apu.kpc,use_physical=True).to(apu.km**2/apu.s**2).value

    # Derivative quantities
    for i in range(len(dr_bin_cents)):
        bin_mask = (rs>=dr_bin_edge[i]) & (rs<dr_bin_edge[i+1])
        n_in_bin = np.sum( bin_mask )
        bin_vol = 4*np.pi/3*(dr_bin_edge[i+1]**3-dr_bin_edge[i]**3)
        dr_nu = n_in_bin/bin_vol
        dr_vr2 = np.mean(orbs.vr(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
        if calculate_pe_with_pot:
            phi[i] = pe_bin_cents[i]
        else:
            phi[i] = np.mean(pe[bin_mask])
        nuvr2[i] = dr_nu*dr_vr2
    dphidr = np.diff(phi)/np.diff(dr_bin_cents)
    dnuvr2dr = np.diff(nuvr2)/np.diff(dr_bin_cents)

    # Non-derivative quantities
    for i in range(len(bin_cents)):
        bin_mask = (rs>=bin_edge[i]) & (rs<bin_edge[i+1])
        n_in_bin = np.sum( bin_mask )
        bin_vol = 4*np.pi/3*(bin_edge[i+1]**3-bin_edge[i]**3)
        nu[i] = n_in_bin/bin_vol
        vr2[i] = np.mean(orbs.vr(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
        vp2[i] = np.mean(orbs.vtheta(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
        vt2[i] = np.mean(orbs.vT(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
    
    # Normalize densities by number of orbits so they're proper number 
    # densities
    nu /= len(orbs)
    dnuvr2dr /= len(orbs)

    if norm_by_galpy_scale_units:
        nu = nu*(ro**3)
        vr2 = vr2/(vo**2)
        vp2 = vp2/(vo**2)
        vt2 = vt2/(vo**2)
        bin_cents = bin_cents/ro
        dphidr = dphidr*ro/(vo**2)
        dnuvr2dr = dnuvr2dr*(ro**4)/(vo**2)

    return dnuvr2dr,dphidr,nu,vr2,vp2,vt2,bin_cents

def calculate_spherical_jeans(orbs,pot,r_range=[0,100],n_bin=10,
    norm_by_galpy_scale_units=False,norm_by_nuvr2_r=True,
    calculate_pe_with_pot=False,return_kinematics=True,ro=ro,vo=vo):
    '''calculate_spherical_jeans:

    Calculate the spherical Jeans equation for a given kinematic sample

    Args:
        orbs (Orbits) - Orbits object containing particles / kinematic sample
        pot (Potential) - Potential object representing the gravitational 
            potential experienced by orbs
        r_range (optional, list) - Range of radii to consider, in kpc 
            [default: [0,100]]
        n_bin (optional, int) - Number of bins to use in calculating Jeans
            equation, note derivative quantities will be calculated with 
            n_bin+1 bins [default: 10]
        norm_by_galpy_scale_units (optional, bool) - If True, normalize the
            Jeans equation by galpy scale units [default: False]
        norm_by_nuvr2_r (optional, bool) - If True, normalize the Jeans equation
            by nu*vr^2/r [default: True]
        calculate_pe_with_pot (optional, bool) - If True, calculate the 
            potential at the bin centers, rather than the mean potential of the 
            orbs in the bin [default: False]
        return_kinematics (optional, bool) - If True, return the kinematics
            used to calculate the Jeans equation [default: True]
        ro (optional, float) - Distance scale in kpc [default: 8.275]
        vo (optional, float) - Velocity scale in km/s [default: 220.]
    
    Returns:
        J (np.ndarray) - Jeans equation, may be normalized
        rs (np.ndarray) - Radii at which Jeans equation is calculated
        qs (tuple) - Tuple of kinematic quantities used to calculate Jeans
            equation, output from calculate_spherical_jeans_quantities, 
            in order: dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs
    '''
    # Compute the 
    qs = calculate_spherical_jeans_quantities(orbs,pot,r_range=r_range,
        n_bin=n_bin,norm_by_galpy_scale_units=norm_by_galpy_scale_units,
        calculate_pe_with_pot=calculate_pe_with_pot,ro=ro,vo=vo)

    dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs = qs

    # Compute the Jeans equation
    J = nu*(dphidr + (2*vr2-vp2-vt2)/rs) + dnuvr2dr

    # Normalize by nu*vr^2/r if desired. Note that this returns the same 
    # answer regardless of whether using physical or galpy units.
    if norm_by_nuvr2_r and not norm_by_galpy_scale_units:
        J = J/(nu*vr2/rs)

    if return_kinematics:
        return J,rs,qs
    else:
        return J,rs

In [ ]:
def plot_jeans_diagnostics(Js,rs,qs,adf=None,r_range=None):

    plot_jeans_sigmas = True
    data_color = 'Black'
    data_linewidth = 2.
    truth_color = 'Red'
    
    _has_adf = adf is not None
    if _has_adf:
        assert r_range is not None
        pot = adf._pot
        denspot = adf._denspot
    else:
        plot_jeans_sigmas = False

    percfunc =  lambda x: np.percentile(np.atleast_2d(x), [16,50,84], axis=0)

    fig = plt.figure(figsize=(12,8))
    gs = fig.add_gridspec(nrows=4,ncols=3)
    axs = np.array([fig.add_subplot(gs[:2,0]),
                    fig.add_subplot(gs[0,1]),
                    fig.add_subplot(gs[1,1]),
                    fig.add_subplot(gs[:2,2]),
                    fig.add_subplot(gs[2:,0]),
                    fig.add_subplot(gs[2,1]),
                    fig.add_subplot(gs[3,1]),
                    fig.add_subplot(gs[2:,2])
                    ])
    # axs = fig.subplots(nrows=2,ncols=3).flatten()

    # J in the first panel
    lJ,mJ,uJ = percfunc(Js)
    axs[0].plot(rs, mJ, color=data_color, linewidth=data_linewidth)
    axs[0].fill_between(rs, lJ, uJ, color='Black', alpha=0.25)
    axs[0].axhline(0, color='Black', linestyle='--', linewidth=0.5)
    axs[0].set_xlim(0,50)
    axs[0].set_xlabel('r [kpc]')
    if norm_by_nuvr2_r:
        axs[0].set_ylabel(r'$J / (\nu \bar{v_{r}^{2}} / r)$')
    else:
        axs[0].set_ylabel('$J$')
    fig.suptitle('Hernquist, scale 10 kpc')

    # Density in the second upper panel
    lnu,mnu,unu = percfunc(qs[2])
    axs[1].plot(rs, mnu, color=data_color, linewidth=data_linewidth)
    if _has_adf:
        denspot_dens = potential.evaluateDensities(denspot,rs*apu.kpc,0)
        denspot_norm = potential.mass(denspot,r_range[1]*apu.kpc)-\
                       potential.mass(denspot,r_range[0]*apu.kpc)
        denspot_dens = (denspot_dens/(denspot_norm)).to(apu.kpc**-3).value
        axs[1].plot(rs, denspot_dens,#*mnu[0]/denspot_dens[0], 
            color=truth_color, linestyle='--')
    axs[1].fill_between(rs, unu, lnu, color='Black', alpha=0.25)
    # axs[1].set_xlim(0,50)
    axs[1].set_xscale('log')
    axs[1].set_yscale('log')
    # axs[1].set_xlabel(r'r [kpc]')
    axs[1].set_ylabel(r'$\nu$')

    # Delta density in the second lower panel
    if _has_adf:
        # dnu = (qs[2] - denspot_dens*mnu[0]/denspot_dens[0])/(denspot_dens*mnu[0]/denspot_dens[0])
        dnu = (qs[2]-denspot_dens)/denspot_dens
        ldnu,mdnu,udnu = percfunc(dnu)
        axs[2].plot(rs, mdnu, color=data_color, linewidth=data_linewidth)
        axs[2].fill_between(rs, udnu, ldnu, color='Black', alpha=0.25)
        axs[2].axhline(0, color='Black', linestyle='--', linewidth=0.5)
        axs[2].set_xscale('log')
        axs[2].set_xlabel(r'r [kpc]')
        axs[2].set_ylabel(r'$\Delta \nu$ [fractional]')

    # Beta in the third panel
    beta = 1 - (qs[4]+qs[5])/(2*qs[3])
    lbeta,mbeta,ubeta = percfunc(beta)
    axs[3].plot(rs, mbeta, color=data_color, linewidth=data_linewidth)
    axs[3].fill_between(rs, ubeta, lbeta, color='Black', alpha=0.25)
    axs[3].axhline(0, color='Black', linestyle='--', linewidth=0.5)
    axs[3].set_xlim(0,50)
    axs[3].set_xlabel(r'r [kpc]')
    axs[3].set_ylabel(r'$\beta$')

    # Radial velocity dispersions in the fourth panel, polar and azimuthal 
    # in the fifth upper/lower panels
    colors = ['DodgerBlue','Crimson','DarkOrange']
    v2_names = [r'$\bar{v_{r}^{2}}$',
                r'$\bar{v_{\phi}^{2}}$',
                r'$\bar{v_{\theta}^{2}}$',]
    for i in range(3):
        for j in range(3):
            lv2,mv2,uv2 = percfunc(qs[j+3])
            if i == j:
                axs[i+4].plot(rs, mv2, color=colors[j], 
                    linewidth=data_linewidth+2, zorder=2)
                axs[i+4].fill_between(rs, uv2, lv2, color=colors[i], alpha=0.25, 
                    zorder=1)
                if plot_jeans_sigmas:
                    mom = [0]*len(rs)
                    for k in range(len(rs)):
                        if i == 0:
                            mom[k] = adf.vmomentdensity(rs[k]*apu.kpc,2,0)
                        elif i in [1,2]:
                            mom[k] = adf.vmomentdensity(rs[k]*apu.kpc,0,2)/2
                        mom[k] /= adf.vmomentdensity(rs[k]*apu.kpc,0,0)
                        mom[k] = mom[k].to_value(apu.km**2/apu.s**2)
                    mom = np.asarray(mom)
                    axs[i+4].plot(rs, mom, color='Black', alpha=1.,
                        linestyle='--', linewidth=1., zorder=3)
            else:
                axs[i+4].plot(rs, mv2, color=colors[j], alpha=1., 
                    linestyle='--', linewidth=1., zorder=3)
        axs[i+4].set_xlim(0,50)
        if i in [0,2]:
            axs[i+4].set_xlabel(r'r [kpc]')
        axs[i+4].set_ylabel(v2_names[i])
        axs[i+4].set_yscale('log')
    
    # dphi/dr in the sixth panel
    ldphidr,mdphidr,udphidr = percfunc(qs[1])
    axs[7].plot(rs, mdphidr, color=data_color, linewidth=data_linewidth)
    axs[7].fill_between(rs, udphidr, ldphidr, color='Black', alpha=0.25)
    if _has_adf:
        negpf = -potential.evaluaterforces(pot,rs*apu.kpc,0).\
            to(apu.km**2/apu.s**2/apu.kpc).value
        axs[7].plot(rs, negpf, color=truth_color, linestyle='--')
    axs[7].set_xlim(0,50)
    axs[7].set_xlabel(r'r [kpc]')
    axs[7].set_ylabel(r'$\mathrm{d}\Phi/\mathrm{d}r$')
    axs[7].set_yscale('log')
    
    return fig,axs

def plot_jeans_time(Js,rs,qs,adf,r_range):
    '''plot_jeans_time:

    Plot the Jeans equation term for a series of times.

    Args:


    Returns:

    '''
    # Define the figure
    fig = plt.figure()
    axs = fig.subplots(1,1)

    # Plot the Jeans equation term
    


In [ ]:
def offset_orbits_cartesian(orbs,vec):
    '''offset_orbits_cartesian:
    
    Offset orbits by a vector in cartesian coordinates
    
    Args:
        orbs (Orbits) - Orbits object containing orbits to offset
        vec (np.ndarray) - 6-vector of [x,y,z,vx,vy,vz] to offset orbits by
             in [kpc...,km/s...]
    
    Returns:
        orbs_offset (Orbits) - Orbits object containing offset orbits
    '''
    ro = orbs._ro
    vo = orbs._vo

    # Get the cartesian positions and velocities
    xs = orbs.x(use_physical=True).to(apu.kpc)
    ys = orbs.y(use_physical=True).to(apu.kpc)
    zs = orbs.z(use_physical=True).to(apu.kpc)
    vxs = orbs.vx(use_physical=True).to(apu.km/apu.s)
    vys = orbs.vy(use_physical=True).to(apu.km/apu.s)
    vzs = orbs.vz(use_physical=True).to(apu.km/apu.s)

    # Offset the orbits
    xs += vec[0]
    ys += vec[1]
    zs += vec[2]
    vxs += vec[3]
    vys += vec[4]
    vzs += vec[5]

    # Create a new Orbits object with the offset orbits
    Rs = (np.sqrt(xs**2+ys**2)).to(apu.kpc)
    phis = (np.arctan2(ys,xs)).to(apu.rad)
    vRs = ((xs*vxs+ys*vys)/Rs).to(apu.km/apu.s)
    vTs = ((xs*vys-ys*vxs)/Rs).to(apu.km/apu.s)

    vxvv = np.vstack((Rs.value/ro,
                      vRs.value/vo,
                      vTs.value/vo,
                      zs.value/ro,
                      vzs.value/vo,
                      phis.value)).T
    
    return orbit.Orbit(vxvv=vxvv,ro=ro,vo=vo)

def plot_6d_cartesian_phase_space(orbs,xrange=[-20,20],vrange=[-200,200]):
    '''plot_6d_phase_space:

    '''
    # Get the cartesian positions and velocities
    xs = orbs.x(use_physical=True).to(apu.kpc).value
    ys = orbs.y(use_physical=True).to(apu.kpc).value
    zs = orbs.z(use_physical=True).to(apu.kpc).value
    vxs = orbs.vx(use_physical=True).to(apu.km/apu.s).value
    vys = orbs.vy(use_physical=True).to(apu.km/apu.s).value
    vzs = orbs.vz(use_physical=True).to(apu.km/apu.s).value

    s=1
    alpha=0.1
    fontsize=10

    fig,axes = plt.subplots(2,3,figsize=(6,4))
    axes = axes.flatten()

    axes[0].scatter(xs,ys,s=s,alpha=alpha)
    axes[0].set_xlabel(r'$x$ [kpc]',fontsize=fontsize)
    axes[0].set_ylabel(r'$y$ [kpc]',fontsize=fontsize)
    axes[0].set_xlim(xrange)
    axes[0].set_ylim(xrange)

    axes[1].scatter(xs,zs,s=s,alpha=alpha)
    axes[1].set_xlabel(r'$x$ [kpc]',fontsize=fontsize)
    axes[1].set_ylabel(r'$z$ [kpc]',fontsize=fontsize)
    axes[1].set_xlim(xrange)
    axes[1].set_ylim(xrange)

    axes[2].scatter(ys,zs,s=s,alpha=alpha)
    axes[2].set_xlabel(r'$y$ [kpc]',fontsize=fontsize)
    axes[2].set_ylabel(r'$z$ [kpc]',fontsize=fontsize)
    axes[2].set_xlim(xrange)
    axes[2].set_ylim(xrange)
    
    axes[3].scatter(vxs,vys,s=s,alpha=alpha)
    axes[3].set_xlabel(r'$v_x$ [km/s]',fontsize=fontsize)
    axes[3].set_ylabel(r'$v_y$ [km/s]',fontsize=fontsize)
    axes[3].set_xlim(vrange)
    axes[3].set_ylim(vrange)

    axes[4].scatter(vxs,vzs,s=s,alpha=alpha)
    axes[4].set_xlabel(r'$v_x$ [km/s]',fontsize=fontsize)
    axes[4].set_ylabel(r'$v_z$ [km/s]',fontsize=fontsize)
    axes[4].set_xlim(vrange)
    axes[4].set_ylim(vrange)
    
    axes[5].scatter(vys,vzs,s=s,alpha=alpha)
    axes[5].set_xlabel(r'$v_y$ [km/s]',fontsize=fontsize)
    axes[5].set_ylabel(r'$v_z$ [km/s]',fontsize=fontsize)
    axes[5].set_xlim(vrange)
    axes[5].set_ylim(vrange)

    for ax in axes:
        ax.set_aspect('equal')
        ax.axhline(0,color='k',ls='--')
        ax.axvline(0,color='k',ls='--')
        ax.tick_params(labelsize=fontsize)

    plt.tight_layout()
    return fig,axes

### Start with a Hernquist embedded in another Hernquist

In [ ]:
n_orbs = int(1e4)

pot = potential.HernquistPotential(amp=1e12*apu.M_sun,a=20*apu.kpc,ro=ro,vo=vo)
denspot = potential.HernquistPotential(amp=1.,a=5*apu.kpc,ro=ro,vo=vo)
edf = df.eddingtondf(pot,denspot=denspot,ro=ro,vo=vo)
orbs = edf.sample(n=n_orbs,return_orbit=True)

In [ ]:
# Create the offset vector
roff = 10*apu.kpc
voff = potential.vcirc(pot,roff)
offset = [roff,0,0,0,voff/2.,0]
orbs_offset = offset_orbits_cartesian(orbs,offset)

In [ ]:
fig,axs = plot_6d_cartesian_phase_space(orbs,xrange=[-20,20],vrange=[-200,200])

In [ ]:
fig,axs = plot_6d_cartesian_phase_space(orbs_offset,xrange=[-20,20],vrange=[-300,300])

### Now integrate some orbits

In [ ]:
tdyn = potential.tdyn(pot,pot.a)
tmax = 5*tdyn
nt = 50
ts = np.linspace(0,tmax.value,nt)*tmax.unit

# Integrate the orbits
orbs.integrate(ts,pot)
orbs_offset.integrate(ts,pot)

In [ ]:
# Show the time evolution of the unperturbed orbits
tplot = [ts[0],ts[-1]]
tplot_title = [r'$t=0$',r'$t=5t_{dyn}$']

for i in range(len(tplot)):
    fig,axs = plot_6d_cartesian_phase_space(orbs(tplot[i]),xrange=[-50,50],
        vrange=[-300,300])
    fig.suptitle('unperturbed orbits '+tplot_title[i],fontsize=12)

In [ ]:
# Show the time evolution of the perturbed orbits
tplot = [ts[0],ts[nt//10],ts[nt//5],ts[-1]]
tplot_title = [r'$t=0$',r'$t=t_{dyn}/2$',r'$t=t_{dyn}$',r'$t=5t_{dyn}$']

for i in range(len(tplot)):
    fig,axs = plot_6d_cartesian_phase_space(orbs_offset(tplot[i]),xrange=[-50,50],
        vrange=[-300,300])
    fig.suptitle('perturbed orbits '+tplot_title[i],fontsize=12)

### Plot the Jeans quantities and diagnostics for a series of times

In [ ]:
nbins = 10
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False

Js = np.zeros((nt,nbins))
qs = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(orbs(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
# for i in range(len(tplot)):
fig,axs = plot_jeans_diagnostics(Js[0:(nt//5)],rs,qs[:,0:(nt//5),:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'unperturbed $t: 0 \rightarrow t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[int(nt-nt//5):],rs,qs[:,int(nt-nt//5):,:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'unperturbed $t: 4t_{dyn} \rightarrow 5t_{dyn}$',fontsize=12)
fig.tight_layout()

In [ ]:
Js_weighted = np.zeros(nt)
for i in range(nt):
    Js_weighted[i] = np.sum(np.abs(Js[i,:]*rs**2))/np.sum(rs**2)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(ts/tdyn,Js_weighted)
ax.set_xlabel(r'$t/t_{dyn}$')
ax.set_ylabel(r'weighted $\langle J \rangle$')
fig.suptitle('unperturbed orbits')
fig.show()

In [ ]:
nbins = 10
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False

Js = np.zeros((nt,nbins))
qs = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(orbs_offset(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
# for i in range(len(tplot)):
fig,axs = plot_jeans_diagnostics(Js[0:(nt//5)],rs,qs[:,0:(nt//5),:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbed orbits $t: 0 \rightarrow t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[int(nt-nt//5):],rs,qs[:,int(nt-nt//5):,:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbed orbits $t: 4t_{dyn} \rightarrow 5t_{dyn}$',fontsize=12)
fig.tight_layout()

In [ ]:
Js_weighted = np.zeros(nt)
for i in range(nt):
    Js_weighted[i] = np.sum(np.abs(Js[i,:]*rs**2))/np.sum(rs**2)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(ts/tdyn,Js_weighted)
ax.set_xlabel(r'$t/t_{dyn}$')
ax.set_ylabel(r'weighted $\langle J \rangle$')

fig.show()

## Try a different perturbation - sample orbits in equilibrium, then suddenly increase the mass of the potential

In [ ]:
n_orbs = int(1e4)
pot = potential.HernquistPotential(amp=1e12*apu.M_sun,a=20*apu.kpc,ro=ro,vo=vo)
pot_perturb = potential.HernquistPotential(amp=4e12*apu.M_sun,a=20*apu.kpc,ro=ro,vo=vo)
denspot = potential.HernquistPotential(amp=1.,a=5*apu.kpc,ro=ro,vo=vo)
edf = df.eddingtondf(pot,denspot=denspot,ro=ro,vo=vo)
orbs = edf.sample(n=n_orbs,return_orbit=True)

print(potential.tdyn(pot,pot.a))
print(potential.tdyn(pot_perturb,pot_perturb.a))

In [ ]:
tdyn = potential.tdyn(pot,pot.a)
tmax = 2*tdyn
nt = 20
ts = np.linspace(0,tmax.value,nt)*tmax.unit

# Integrate the orbits
orbs.integrate(ts,pot)
orbs_perturb = orbs(ts[-1])
orbs_perturb.integrate(ts,pot_perturb)

In [ ]:
nbins = 10
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False

Js = np.zeros((nt,nbins))
qs = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(orbs(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

Js_perturb = np.zeros((nt,nbins))
qs_perturb = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(orbs_perturb(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js_perturb[i,:] = _J
    for j in range(len(_qs)):
        qs_perturb[j,i,:] = _qs[j]

In [ ]:
# for i in range(len(tplot)):
fig,axs = plot_jeans_diagnostics(Js[0:nt//2],rs,qs[:,0:nt//2,:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'unperturbed $t: 0 \rightarrow t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js_perturb[0:nt//2],rs,
    qs_perturb[:,0:nt//2,:],adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbed $t: 0 \rightarrow 2t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js_perturb[(nt//2):],rs,
    qs_perturb[:,(nt//2):,:],adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbed $t: 2t_{dyn} \rightarrow 4t_{dyn}$',fontsize=12)
fig.tight_layout()

In [ ]:
Js_weighted = np.zeros(nt)
for i in range(nt):
    Js_weighted[i] = np.sum(np.abs(Js[i,:]*rs**2))/np.sum(rs**2)
Js_weighted_perturb = np.zeros(nt)
for i in range(nt):
    Js_weighted_perturb[i] = np.sum(np.abs(Js_perturb[i,:]*rs**2))/np.sum(rs**2)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(ts/tdyn,Js_weighted,color='DodgerBlue')
ax.plot(2*ts/tdyn+(ts/tdyn)[-1],Js_weighted_perturb,color='DarkOrange')
ax.set_xlabel(r'$t/t_{dyn}$')
ax.set_ylabel(r'weighted $\langle J \rangle$')

fig.show()

### Do the same experiment but now use the Dehnen Wrapper to make the transformation happen smoothly

In [ ]:
n_orbs = int(1e4)
pot = potential.HernquistPotential(amp=1e12*apu.M_sun,a=20*apu.kpc,ro=ro,vo=vo)
tdyn_init = potential.tdyn(pot, pot.a)
pot_perturb = potential.HernquistPotential(amp=4e12*apu.M_sun,a=20*apu.kpc,ro=ro,vo=vo)
pot_perturb_diff = potential.HernquistPotential(amp=3e12*apu.M_sun, a=20*apu.kpc, ro=ro, vo=vo)
denspot = potential.HernquistPotential(amp=1.,a=10*apu.kpc,ro=ro,vo=vo)
edf = df.eddingtondf(pot,denspot=denspot,ro=ro,vo=vo)
orbs = edf.sample(n=n_orbs,return_orbit=True)

print(potential.tdyn(pot,pot.a))
print(potential.tdyn(pot_perturb,pot_perturb.a))

In [ ]:
# Create the steadily evolving potential
tform = 1*tdyn_init
tsteady = 5*tdyn_init
pot_grow = potential.DehnenSmoothWrapperPotential(pot=pot_perturb_diff, 
    tform=tform, tsteady=tsteady, decay=False)

In [ ]:
def dehnen_amp(t,tform,tsteady,decay=False):
    if t < tform:
        xi = -1
    elif t >= tform and t <= (tform+tsteady):
        xi = 2*((t-tform)/tsteady)-1
    elif t > (tform+tsteady):
        xi = 1
    else:
        print('Something went wrong')
    amp = 3*xi**5/16 - 5*xi**3/8 + 15*xi/16 + 0.5
    if decay:
        return 1-amp
    else:
        return amp

fig = plt.figure()
ax = fig.add_subplot(111)

ts_plot = np.linspace(0.,1.,num=100)*6*tdyn_init.value
amp = np.array([dehnen_amp(t,tform.value,tsteady.value,decay=False) for t in ts_plot])

ax.plot(ts_plot/tdyn_init.value, amp)
ax.set_xlabel('time [initial dynamical]')
ax.set_ylabel('Amp')

fig.show()

In [ ]:
nt = 600
ts = np.linspace(0., 6*tdyn_init.value, num=nt)*apu.Gyr
orbs.integrate(ts, [pot,pot_grow])

In [ ]:
nbins = 10
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False

Js = np.zeros((nt,nbins))
qs = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(orbs(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
# for i in range(len(tplot)):
fig,axs = plot_jeans_diagnostics(Js[0:nt//6],rs,qs[:,0:nt//6,:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'unperturbed $t: 0 \rightarrow t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[nt//6:(3*nt)//6],rs,
    qs[:,nt//6:(3*nt)//6,:],adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbation beginning $t: t_{dyn} \rightarrow 3t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[(4*nt)//6:],rs,
    qs[:,(4*nt)//6:,:],adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbation ending $t: 4t_{dyn} \rightarrow 6t_{dyn}$',fontsize=12)
fig.tight_layout()

In [ ]:
Js_weighted = np.zeros(nt)
for i in range(nt):
    Js_weighted[i] = np.sum(np.abs(Js[i,:]*rs**2))/np.sum(rs**2)

fig = plt.figure(figsize=(6,4))
gs = matplotlib.gridspec.GridSpec(nrows=4, ncols=1, figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1:])

amp = np.array([dehnen_amp(t,tform,tsteady,decay=False) for t in ts])
ax1.plot(ts/tdyn_init, amp, color='DodgerBlue')
ax1.set_ylabel('amp')
ax1.tick_params(labelbottom=False)

ax2.plot(ts/tdyn_init,Js_weighted,color='DodgerBlue')
ax2.set_xlabel(r'$t/t_{dyn}$')
ax2.set_ylabel(r'weighted $\langle J \rangle$')

fig.show()

## Now do a complicated experiment where you embed an equilibrium Hernquist model in a moving object potential which is being accreted

In [ ]:
## First try and create a moving object potential which experiences dynamical friction

# Host potential
pot_mass = 1e12*apu.M_sun
pot_scale = 20*apu.kpc
pot = potential.HernquistPotential(amp=2*pot_mass, a=pot_scale, ro=ro,vo=vo) # host potential

# Satellite potential
sat_mass = 1e11*apu.M_sun
sat_scale = 10*apu.kpc
sat_rhm = (1+np.sqrt(2))*sat_scale
satpot = potential.HernquistPotential(amp=2*sat_mass, a=sat_scale, ro=ro, vo=vo)

# Satellite density potential
sat_dens_scale = 5*apu.kpc
satdenspot = potential.HernquistPotential(amp=1., a=sat_dens_scale, ro=ro, vo=vo)
satdf = df.eddingtondf(pot=satpot, denspot=satdenspot, ro=ro, vo=vo)

# The dynamical friction potential
dynfricpot = potential.ChandrasekharDynamicalFrictionForce(GMs=sat_mass, rhm=sat_rhm, dens=pot, ro=ro, vo=vo)

In [ ]:
# Start the orbit at twice the scale radius and let it evolve
R_start = 2*pot_scale
vcirc_start = potential.vcirc(pot, R_start)
tdyn_start = potential.tdyn(pot, R_start)
vxvv = [R_start.value/ro, 0., vcirc_start.value/vo, 0., 0., 0.]
o_dynfric = orbit.Orbit(vxvv, ro=ro, vo=vo)

nt = 1000
ts = np.linspace(0., 3*tdyn_start.value, num=nt)*apu.Gyr
o_dynfric.integrate(ts,[pot,dynfricpot])

satmop = potential.MovingObjectPotential(o_dynfric, pot=satpot, ro=ro, vo=vo)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(o_dynfric.x(ts), o_dynfric.y(ts))

ax.set_xlabel('X [kpc]')
ax.set_ylabel('Y [kpc]')
ax.set_xlim(-45,45)
ax.set_ylim(-45,45)

fig.show()

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(ts/tdyn_start, o_dynfric.r(ts))

ax.set_xlabel('t [dynamical]')
ax.set_ylabel('r [kpc]')
ax.set_xlim(0,3)
ax.set_ylim(0,45)

fig.show()

In [ ]:
# Create and offset the satellite orbits
satorbs = satdf.sample(n=10000)
offset_vec = [R_start,0.,0.,0.,vcirc_start,0.]
satorbs = offset_orbits_cartesian(satorbs, offset_vec)

nt = 600
ts = np.linspace(0., 3*tdyn_start.value, num=nt)*apu.Gyr
satorbs.integrate(ts, [satmop,pot])

In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(nrows=2, ncols=3).flatten()
n_axs = 6

ts_indx = [0, 100, 200, 300, 400, 500]
ts_strs = [r'$t=0$', r'$t=0.5t_{dyn}$', r'$t=t_{dyn}$', r'$t=1.5t_{dyn}$', r'$t=2t_{dyn}$', r'$t=2.5t_{dyn}$']

for i in range(n_axs):
    ax = axs[i]
    _t = ts[ts_indx[i]]
    
    ax.scatter(satorbs.x(_t), satorbs.y(_t), alpha=0.1, s=1., zorder=2, 
               color='Black')
    ax.plot(o_dynfric.x(ts), o_dynfric.y(ts), alpha=0.5, linewidth=1., 
            color='DodgerBlue', zorder=3)
    ax.scatter(o_dynfric.x(_t), o_dynfric.y(_t), s=16, marker='x', 
               color='DodgerBlue', zorder=4)
    
    ax.set_xlabel('X [kpc]')
    ax.set_ylabel('Y [kpc]')
    ax.set_xlim(-45,45)
    ax.set_ylim(-45,45)
    
    ax.set_title(ts_strs[i])

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(nrows=2, ncols=3).flatten()
n_axs = 6

ts_indx = [0, 100, 200, 300, 400, 500]
ts_strs = [r'$t=0$', r'$t=0.5t_{dyn}$', r'$t=t_{dyn}$', r'$t=1.5t_{dyn}$', r'$t=2t_{dyn}$', r'$t=2.5t_{dyn}$']

for i in range(n_axs):
    ax = axs[i]
    _t = ts[ts_indx[i]]
    
    ax.hist(satorbs.r(_t).value, bins=25, range=(0,100), density=True, 
        histtype='step')
    
    ax.set_xlabel('r [kpc]')
    ax.set_ylabel('p(r)')
    ax.set_xlim(0,100)
    ax.set_ylim(0,0.075)
    
    ax.set_title(ts_strs[i])

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(nrows=2, ncols=3).flatten()
n_axs = 6

ts_indx = [0, 100, 200, 300, 400, 500]
ts_strs = [r'$t=0$', r'$t=0.5t_{dyn}$', r'$t=t_{dyn}$', r'$t=1.5t_{dyn}$', r'$t=2t_{dyn}$', r'$t=2.5t_{dyn}$']

for i in range(n_axs):
    ax = axs[i]
    _t = ts[ts_indx[i]]
    
    hist,bin_edges = np.histogram(satorbs.r(_t).value, bins=25, range=(0,100))
    bin_cents = (bin_edges[1:]+bin_edges[:-1])/2
    shell_vol = (4*np.pi/3)*(bin_edges[1:]**3-bin_edges[:-1])
    dens = hist/shell_vol
    
    ax.plot(bin_cents, dens, color='DodgerBlue')
    
    # Comparison with fiducial power laws
    alphas = [0.5,1.5,2.5,3.5,4.5,]
    rs = np.linspace(bin_cents[0],100,num=101)*apu.kpc
    for alpha in alphas:
        fpot = potential.PowerSphericalPotential(alpha=alpha)
        fdens = potential.evaluateDensities(fpot,rs,0)
        ax.plot(rs.value, fdens*(dens[0]/fdens[0]), color='Red', linestyle='dashed', 
                linewidth=0.5, alpha=0.5)
        
    
    ax.set_xlabel('r [kpc]')
    ax.set_ylabel('number density')
    ax.set_xlim(1,100)
    # ax.set_ylim(0,0.075)
    ax.set_yscale('log')
    ax.set_xscale('log')
    
    ax.set_title(ts_strs[i])

fig.tight_layout()
fig.show()

In [ ]:
nbins = 10
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False

Js = np.zeros((nt,nbins))
qs = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(satorbs(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
# for i in range(len(tplot)):
fig,axs = plot_jeans_diagnostics(Js[0:nt//3],rs,qs[:,0:nt//3,:],
    adf=None,r_range=samp_r_range)
fig.suptitle(r'$t: 0 \rightarrow t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[nt//3:(2*nt)//3],rs,
    qs[:,nt//3:(2*nt)//3,:],adf=None,r_range=samp_r_range)
fig.suptitle(r'$t: t_{dyn} \rightarrow 2t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[(2*nt)//3:],rs,
    qs[:,(2*nt)//3:,:],adf=None,r_range=samp_r_range)
fig.suptitle(r'$t: 2t_{dyn} \rightarrow 3t_{dyn}$',fontsize=12)
fig.tight_layout()

In [ ]:
Js_weighted = np.zeros(nt)
for i in range(nt):
    Js_weighted[i] = np.sum(np.abs(Js[i,:]*rs**2))/np.sum(rs**2)

fig = plt.figure()#figsize=(6,4))
# gs = matplotlib.gridspec.GridSpec(nrows=4, ncols=1, figure=fig)
#ax1 = fig.add_subplot(gs[0])
#ax2 = fig.add_subplot(gs[1:])
ax2 = fig.add_subplot(111)

# amp = np.array([dehnen_amp(t,tform,tsteady,decay=False) for t in ts])
# ax1.plot(ts/tdyn_start, amp, color='DodgerBlue')
# ax1.set_ylabel('amp')
# ax1.tick_params(labelbottom=False)

ax2.plot(ts/tdyn_start,Js_weighted,color='DodgerBlue')
ax2.set_xlabel(r'$t/t_{dyn}$')
ax2.set_ylabel(r'weighted $\langle J \rangle$')

fig.show()

### Build on the experiment by having the potential dissolve with time

In [ ]:
# Create the dissolving satellite potential
tform = 0*tdyn_start
tsteady = 3.*tdyn_start
satmop_dissolve = potential.DehnenSmoothWrapperPotential(pot=satmop, tform=tform, tsteady=tsteady, decay=True)

# Create and offset the satellite orbits
satorbs_dissolve = satdf.sample(n=10000)
offset_vec = [R_start,0.,0.,0.,vcirc_start,0.]
satorbs_dissolve = offset_orbits_cartesian(satorbs_dissolve, offset_vec)

In [ ]:
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

amp = np.array([dehnen_amp(t,tform,tsteady,decay=True) for t in ts])
ax1.plot(ts/tdyn_start, amp, color='DodgerBlue')
ax1.set_ylabel('amp')
ax1.set_xlabel('t [dynamical]')
# ax1.tick_params(labelbottom=False)

ax2.plot(ts/tdyn_start, o_dynfric.r(ts))
ax2.set_xlabel('t [dynamical]')
ax2.set_ylabel('r [kpc]')
ax2.set_xlim(0,3)
ax2.set_ylim(0,45)

fig.show()

In [ ]:
nt = 600
ts = np.linspace(0., 3*tdyn_start.value, num=nt)*apu.Gyr
satorbs_dissolve.integrate(ts, [satmop_dissolve,pot])

In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(nrows=2, ncols=3).flatten()
n_axs = 6

ts_indx = [0, 100, 200, 300, 400, 500]
ts_strs = [r'$t=0$', r'$t=0.5t_{dyn}$', r'$t=t_{dyn}$', r'$t=1.5t_{dyn}$', r'$t=2t_{dyn}$', r'$t=2.5t_{dyn}$']

for i in range(n_axs):
    ax = axs[i]
    _t = ts[ts_indx[i]]
    
    ax.scatter(satorbs_dissolve.x(_t), satorbs_dissolve.y(_t), alpha=0.1, s=1., zorder=2, color='Black')
    ax.plot(o_dynfric.x(ts), o_dynfric.y(ts), alpha=0.5, linewidth=1., color='DodgerBlue', zorder=3)
    ax.scatter(o_dynfric.x(_t), o_dynfric.y(_t), marker='x', s=16, color='DodgerBlue', zorder=4)
    
    ax.set_xlabel('X [kpc]')
    ax.set_ylabel('Y [kpc]')
    ax.set_xlim(-45,45)
    ax.set_ylim(-45,45)
    
    ax.set_title(ts_strs[i])

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(nrows=2, ncols=3).flatten()
n_axs = 6

ts_indx = [0, 100, 200, 300, 400, 500]
ts_strs = [r'$t=0$', r'$t=0.5t_{dyn}$', r'$t=t_{dyn}$', r'$t=1.5t_{dyn}$', r'$t=2t_{dyn}$', r'$t=2.5t_{dyn}$']

for i in range(n_axs):
    ax = axs[i]
    _t = ts[ts_indx[i]]
    
    ax.hist(satorbs_dissolve.r(_t).value, bins=25, range=(0,100), density=True, 
        histtype='step')
    
    ax.set_xlabel('r [kpc]')
    ax.set_ylabel('p(r)')
    ax.set_xlim(0,100)
    ax.set_ylim(0,0.075)
    
    ax.set_title(ts_strs[i])

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure(figsize=(12,6))
axs = fig.subplots(nrows=2, ncols=3).flatten()
n_axs = 6

ts_indx = [0, 100, 200, 300, 400, 500]
ts_strs = [r'$t=0$', r'$t=0.5t_{dyn}$', r'$t=t_{dyn}$', r'$t=1.5t_{dyn}$', r'$t=2t_{dyn}$', r'$t=2.5t_{dyn}$']

for i in range(n_axs):
    ax = axs[i]
    _t = ts[ts_indx[i]]
    
    hist,bin_edges = np.histogram(satorbs_dissolve.r(_t).value, bins=25, range=(0,100))
    bin_cents = (bin_edges[1:]+bin_edges[:-1])/2
    shell_vol = (4*np.pi/3)*(bin_edges[1:]**3-bin_edges[:-1])
    dens = hist/shell_vol
    
    ax.plot(bin_cents, dens, color='DodgerBlue')
    
    # Comparison with fiducial power laws
    alphas = [0.5,1.5,2.5,3.5,4.5,]
    rs = np.linspace(bin_cents[0],100,num=101)*apu.kpc
    for alpha in alphas:
        fpot = potential.PowerSphericalPotential(alpha=alpha)
        fdens = potential.evaluateDensities(fpot,rs,0)
        ax.plot(rs.value, fdens*(dens[0]/fdens[0]), color='Red', linestyle='dashed', 
                linewidth=0.5, alpha=0.5)
        
    
    ax.set_xlabel('r [kpc]')
    ax.set_ylabel('number density')
    ax.set_xlim(1,100)
    # ax.set_ylim(0,0.075)
    ax.set_yscale('log')
    ax.set_xscale('log')
    
    ax.set_title(ts_strs[i])

fig.tight_layout()
fig.show()

In [ ]:
nbins = 10
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False

Js = np.zeros((nt,nbins))
qs = np.zeros((7,nt,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

for i in range(nt):
    _J,rs,_qs = calculate_spherical_jeans(satorbs_dissolve(ts[i]), pot, 
        r_range=bin_r_range, n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
# for i in range(len(tplot)):
fig,axs = plot_jeans_diagnostics(Js[0:nt//6],rs,qs[:,0:nt//6,:],
    adf=edf,r_range=samp_r_range)
fig.suptitle(r'unperturbed $t: 0 \rightarrow t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[nt//6:(3*nt)//6],rs,
    qs[:,nt//6:(3*nt)//6,:],adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbation beginning $t: t_{dyn} \rightarrow 3t_{dyn}$',fontsize=12)
fig.tight_layout()

fig,axs = plot_jeans_diagnostics(Js[(4*nt)//6:],rs,
    qs[:,(4*nt)//6:,:],adf=edf,r_range=samp_r_range)
fig.suptitle(r'perturbation ending $t: 4t_{dyn} \rightarrow 6t_{dyn}$',fontsize=12)
fig.tight_layout()

In [ ]:
Js_weighted = np.zeros(nt)
for i in range(nt):
    Js_weighted[i] = np.sum(np.abs(Js[i,:]*rs**2))/np.sum(rs**2)

fig = plt.figure(figsize=(6,4))
gs = matplotlib.gridspec.GridSpec(nrows=4, ncols=1, figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1:])

amp = np.array([dehnen_amp(t,tform,tsteady,decay=True) for t in ts])
ax1.plot(ts/tdyn_init, amp, color='DodgerBlue')
ax1.set_ylabel('amp')
ax1.tick_params(labelbottom=False)

ax2.plot(ts/tdyn_init,Js_weighted,color='DodgerBlue')
ax2.set_xlabel(r'$t/t_{dyn}$')
ax2.set_ylabel(r'weighted $\langle J \rangle$')

fig.show()